## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import librosa
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, f1_score

from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore') 

C:\Users\hist\miniconda3\envs\ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Hyperparameters

In [2]:
EPOCHS = 1000
LR = 1e-1
SR = 16000
SEED = 42
N_MFCC = 128
BATCH = 256
device = 'cuda'
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED) # Seed 고정

## Data Pre-processing

In [3]:
path = 'open/'
train_df = pd.read_csv(path+'train.csv') # 모두 정상 Sample
test_df = pd.read_csv(path+'test.csv')

In [4]:
train_df

,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE,LABEL
0,TRAIN_0000,./train/TRAIN_0000.wav,2,0
1,TRAIN_0001,./train/TRAIN_0001.wav,0,0
2,TRAIN_0002,./train/TRAIN_0002.wav,0,0
3,TRAIN_0003,./train/TRAIN_0003.wav,2,0
4,TRAIN_0004,./train/TRAIN_0004.wav,2,0
...,...,...,...,...
1274,TRAIN_1274,./train/TRAIN_1274.wav,2,0
1275,TRAIN_1275,./train/TRAIN_1275.wav,2,0
1276,TRAIN_1276,./train/TRAIN_1276.wav,2,0
1277,TRAIN_1277,./train/TRAIN_1277.wav,2,0


In [5]:
def get_mfcc_feature(df):
    features = []
    features2 = []
    for path in tqdm(df['SAMPLE_PATH']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=SR)
        
        # melspectrogram
        mels = librosa.feature.melspectrogram(y, sr=sr, n_mels=N_MFCC)
        mels = librosa.power_to_db(mels, ref=np.max)
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=N_MFCC)
        
        y_feature2 = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mels:
            y_feature2.append(np.mean(e))
        features2.append(y_feature2)
        
        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)
    return features, features2

In [6]:
%%time

train_features, train_features2 = get_mfcc_feature(train_df)
test_features, test_features2 = get_mfcc_feature(test_df)

100%|██████████████████████████████████████████████████████████████████████████████| 1514/1514 [00:32<00:00, 47.00it/s]

CPU times: total: 1min 22s
Wall time: 59.2 s


In [7]:
# 반복문으로 전체 컬럼명 변경하기
def rename(df):
    flag = 0
    for col_name in df.columns:
        if col_name == 0:
            flag = 1
        if flag == 1:
            df.rename(columns = {col_name : 128+col_name}, inplace = True)
    return df

In [8]:
tmp = pd.concat([train_df, pd.DataFrame(train_features)], axis=1)
tmp = rename(tmp)
tmp = pd.concat([tmp, pd.DataFrame(train_features2)], axis=1)

test = pd.concat([test_df, pd.DataFrame(test_features)], axis=1)
test = rename(test)
test = pd.concat([test, pd.DataFrame(test_features2)], axis=1)

scaler = MinMaxScaler()
tmp.iloc[:,range(4,len(tmp.columns))] = scaler.fit_transform(tmp.iloc[:,range(4,len(tmp.columns))])
test.iloc[:,range(3,len(test.columns))] = scaler.transform(test.iloc[:,range(3,len(test.columns))])
tmp

,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE,LABEL,128,129,130,131,132,133,...,118,119,120,121,122,123,124,125,126,127
0,TRAIN_0000,./train/TRAIN_0000.wav,2,0,0.952273,0.284621,0.295843,0.363586,0.357530,0.294327,...,0.833846,0.842291,0.818590,0.817896,0.799984,0.799645,0.811261,0.816584,0.825692,0.805742
1,TRAIN_0001,./train/TRAIN_0001.wav,0,0,0.081981,0.935459,0.514880,0.613406,0.691659,0.825306,...,0.148846,0.144862,0.149996,0.165012,0.156853,0.160930,0.163661,0.162874,0.158788,0.084692
2,TRAIN_0002,./train/TRAIN_0002.wav,0,0,0.240260,0.664368,0.724483,0.354632,0.625930,0.695975,...,0.454258,0.520886,0.454087,0.465858,0.423989,0.497721,0.504808,0.502295,0.487844,0.409240
3,TRAIN_0003,./train/TRAIN_0003.wav,2,0,0.943729,0.295295,0.312391,0.371455,0.329344,0.268686,...,0.808817,0.827032,0.815166,0.787913,0.779337,0.781981,0.771998,0.779565,0.796365,0.798277
4,TRAIN_0004,./train/TRAIN_0004.wav,2,0,0.949608,0.188729,0.179787,0.154144,0.007228,0.055841,...,0.877542,0.909439,0.894097,0.863824,0.849838,0.867351,0.865360,0.861068,0.868706,0.875541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,TRAIN_1274,./train/TRAIN_1274.wav,2,0,0.964989,0.096119,0.129592,0.222531,0.039493,0.075616,...,0.857574,0.875432,0.862135,0.853636,0.852105,0.854758,0.852087,0.851688,0.860304,0.881918
1275,TRAIN_1275,./train/TRAIN_1275.wav,2,0,0.959506,0.283203,0.293961,0.390142,0.322176,0.239990,...,0.842018,0.853391,0.822730,0.825484,0.822317,0.809976,0.820805,0.835689,0.836974,0.819587
1276,TRAIN_1276,./train/TRAIN_1276.wav,2,0,0.930908,0.223856,0.210020,0.151861,0.134018,0.149416,...,0.918239,0.932157,0.898576,0.905386,0.894843,0.879353,0.900221,0.910439,0.903620,0.895840
1277,TRAIN_1277,./train/TRAIN_1277.wav,2,0,0.932890,0.247222,0.265837,0.223214,0.170546,0.141445,...,0.922436,0.946884,0.920099,0.916849,0.900980,0.903257,0.919087,0.922190,0.920950,0.932029


In [9]:
tmp.dropna()

,SAMPLE_ID,SAMPLE_PATH,FAN_TYPE,LABEL,128,129,130,131,132,133,...,118,119,120,121,122,123,124,125,126,127
0,TRAIN_0000,./train/TRAIN_0000.wav,2,0,0.952273,0.284621,0.295843,0.363586,0.357530,0.294327,...,0.833846,0.842291,0.818590,0.817896,0.799984,0.799645,0.811261,0.816584,0.825692,0.805742
1,TRAIN_0001,./train/TRAIN_0001.wav,0,0,0.081981,0.935459,0.514880,0.613406,0.691659,0.825306,...,0.148846,0.144862,0.149996,0.165012,0.156853,0.160930,0.163661,0.162874,0.158788,0.084692
2,TRAIN_0002,./train/TRAIN_0002.wav,0,0,0.240260,0.664368,0.724483,0.354632,0.625930,0.695975,...,0.454258,0.520886,0.454087,0.465858,0.423989,0.497721,0.504808,0.502295,0.487844,0.409240
3,TRAIN_0003,./train/TRAIN_0003.wav,2,0,0.943729,0.295295,0.312391,0.371455,0.329344,0.268686,...,0.808817,0.827032,0.815166,0.787913,0.779337,0.781981,0.771998,0.779565,0.796365,0.798277
4,TRAIN_0004,./train/TRAIN_0004.wav,2,0,0.949608,0.188729,0.179787,0.154144,0.007228,0.055841,...,0.877542,0.909439,0.894097,0.863824,0.849838,0.867351,0.865360,0.861068,0.868706,0.875541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,TRAIN_1274,./train/TRAIN_1274.wav,2,0,0.964989,0.096119,0.129592,0.222531,0.039493,0.075616,...,0.857574,0.875432,0.862135,0.853636,0.852105,0.854758,0.852087,0.851688,0.860304,0.881918
1275,TRAIN_1275,./train/TRAIN_1275.wav,2,0,0.959506,0.283203,0.293961,0.390142,0.322176,0.239990,...,0.842018,0.853391,0.822730,0.825484,0.822317,0.809976,0.820805,0.835689,0.836974,0.819587
1276,TRAIN_1276,./train/TRAIN_1276.wav,2,0,0.930908,0.223856,0.210020,0.151861,0.134018,0.149416,...,0.918239,0.932157,0.898576,0.905386,0.894843,0.879353,0.900221,0.910439,0.903620,0.895840
1277,TRAIN_1277,./train/TRAIN_1277.wav,2,0,0.932890,0.247222,0.265837,0.223214,0.170546,0.141445,...,0.922436,0.946884,0.920099,0.916849,0.900980,0.903257,0.919087,0.922190,0.920950,0.932029


In [10]:
cols = tmp.columns.drop(['SAMPLE_ID', 'SAMPLE_PATH', 'LABEL'])
cols

Index(['FAN_TYPE',        128,        129,        130,        131,        132,
              133,        134,        135,        136,
       ...
              118,        119,        120,        121,        122,        123,
              124,        125,        126,        127],
      dtype='object', length=257)

## Model Fit

In [11]:
class CustomDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = df['LABEL'].values
        self.df = df[cols].values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x).reshape(-1,1), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x).reshape(-1,1)
        
    def __len__(self):
        return len(self.df)

In [12]:
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False
        self.min_loss = np.inf

    def __call__(self, train_loss, validation_loss=None):
        if train_loss < self.min_loss:
            self.counter = 0
            self.min_loss = train_loss
            print(f'counter : set 0 min loss : {self.min_loss}')
        elif train_loss > self.min_loss:
            self.counter += 1
            print(f'counter : {self.counter}')
        if self.counter >= self.tolerance:  
            self.early_stop = True

In [13]:
train_df, val_df = train_test_split(tmp, test_size=0.1, random_state=SEED)

In [14]:
train_dataset = CustomDataset(df=train_df, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True)

val_dataset = CustomDataset(df = val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH, shuffle=False)

In [15]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.early_stopping = EarlyStopping(tolerance=100, min_delta=10)
        
        # Loss Function
        self.criterion = nn.KLDivLoss(reduction='batchmean', log_target=True).to(self.device)
        # self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self):
        self.model.to(self.device)
        best_score = float('-inf')
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                self.optimizer.zero_grad()
                x = x.float().to(self.device)
                _x = self.model(x)

                log_target = F.log_softmax(_x, dim=1)
                log_input = F.log_softmax(x.reshape(-1,257), dim=1)
                
                loss = self.criterion(log_input, log_target)
                loss.backward()
                self.optimizer.step()
                
                train_loss.append(loss.item())
            
            score = self.validation(self.model)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')
            
            if best_score < score:
                best_score = score
                torch.save(model.state_dict(), './best_model.pth', _use_new_zipfile_serialization=False)
            # torch.save(model.state_dict(), './model.pth', _use_new_zipfile_serialization=False)
            # early stopping
            self.early_stopping(np.mean(train_loss))
            if self.early_stopping.early_stop:
                print("early_stopping:", epoch)
                break
            
            if self.scheduler is not None:
                self.scheduler.step(score)
                            
    def validation(self, eval_model, thr=0.999):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)
                _x = self.model(x)
                
                log_target = F.log_softmax(_x, dim=1)
                log_input = F.log_softmax(x.reshape(-1,257), dim=1)
                
                diff = cos(log_input, log_target).cpu().tolist()
                # print(diff)
                batch_pred = np.where(np.array(diff) > thr, 0, 1).tolist()

                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

In [16]:
class ResConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResConv, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv1d(in_channels = in_channels, out_channels = out_channels, 
                                 kernel_size = 1, stride = 1),
            nn.BatchNorm1d(num_features = out_channels),
            nn.GELU(),
        )
        self.conv_skip = nn.Sequential(
            nn.Conv1d(in_channels = in_channels, out_channels = out_channels, 
                                 kernel_size = 1, stride = 1),
            nn.BatchNorm1d(num_features = out_channels),
        )

    def forward(self, x):
        return self.conv_block(x) + self.conv_skip(x)

class Upsample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Upsample, self).__init__()

        self.upsample = nn.ConvTranspose1d(
            in_channels, out_channels, kernel_size=1, stride=1
        )

    def forward(self, x):
        return self.upsample(x)
    
class Squeeze_Excite_Block(nn.Module):
    def __init__(self, channel, reduction=16):
        super(Squeeze_Excite_Block, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class ASPP(nn.Module):
    def __init__(self, in_dims, out_dims, rate=[6, 12, 18]):
        super(ASPP, self).__init__()

        self.aspp_block1 = nn.Sequential(
            nn.Conv1d(
                in_dims, out_dims, kernel_size = 1, stride = 1
            ),
            nn.GELU(),
            nn.BatchNorm1d(out_dims),
        )
        self.aspp_block2 = nn.Sequential(
            nn.Conv1d(
                in_dims, out_dims, kernel_size = 1, stride = 1
            ),
            nn.GELU(),
            nn.BatchNorm1d(out_dims),
        )
        self.aspp_block3 = nn.Sequential(
            nn.Conv1d(
                in_dims, out_dims, kernel_size = 1, stride = 1
            ),
            nn.GELU(),
            nn.BatchNorm1d(out_dims),
        )

        self.output = nn.Conv1d(len(rate) * out_dims, out_dims, 1)
        self._init_weights()

    def forward(self, x):
        x1 = self.aspp_block1(x)
        x2 = self.aspp_block2(x)
        x3 = self.aspp_block3(x)
        out = torch.cat([x1, x2, x3], dim=1)
        return self.output(out)

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class Upsample_(nn.Module):
    def __init__(self, scale=2):
        super(Upsample_, self).__init__()

        self.upsample = nn.Upsample(mode="bilinear", scale_factor=scale)

    def forward(self, x):
        return self.upsample(x)

class AttentionBlock(nn.Module):
    def __init__(self, input_encoder, input_decoder, output_dim):
        super(AttentionBlock, self).__init__()

        self.conv_encoder = nn.Sequential(
            nn.BatchNorm1d(input_encoder),
            nn.GELU(),
            nn.Conv1d(input_encoder, output_dim, 3, padding=1),
            nn.MaxPool1d(1, 1),
        )

        self.conv_decoder = nn.Sequential(
            nn.BatchNorm1d(input_decoder),
            nn.GELU(),
            nn.Conv1d(input_decoder, output_dim, 3, padding=1),
        )

        self.conv_attn = nn.Sequential(
            nn.BatchNorm1d(output_dim),
            nn.ReLU(),
            nn.Conv1d(output_dim, 1, 1),
        )

    def forward(self, x1, x2):
        out = self.conv_encoder(x1) + self.conv_decoder(x2)
        out = self.conv_attn(out)
        return out * x2

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.input_layer = nn.Sequential(
            nn.Conv1d(in_channels = 257, out_channels = 512, 
                                 kernel_size = 1, stride = 1),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Conv1d(in_channels = 512, out_channels = 512, 
                                 kernel_size = 1, stride = 1),
        )
        
        self.input_skip = nn.Sequential(
            nn.Conv1d(in_channels = 257, out_channels = 512, 
                                 kernel_size = 1, stride = 1)
        )
        self.squeeze_excite1 = Squeeze_Excite_Block(512)

        self.residual_conv1 = ResConv(512, 1024)

        self.squeeze_excite2 = Squeeze_Excite_Block(1024)

        self.residual_conv2 = ResConv(1024, 2048)

        self.squeeze_excite3 = Squeeze_Excite_Block(2048)

        self.residual_conv3 = ResConv(2048, 4096)

        self.aspp_bridge = ASPP(4096, 4096)

        self.attn1 = AttentionBlock(2048, 4096, 4096)
        self.upsample1 = Upsample_(1)
        self.up_residual_conv1 = ResConv(4096 + 2048, 2048)

        self.attn2 = AttentionBlock(1024, 2048, 2048)
        self.upsample2 = Upsample_(1)
        self.up_residual_conv2 = ResConv(2048 + 1024, 2048)

        self.attn3 = AttentionBlock(512, 2048, 2048)
        self.upsample3 = Upsample_(1)
        self.up_residual_conv3 = ResConv(2048 + 512, 1024)

        self.aspp_out = ASPP(2048, 512)

        self.flat = nn.Flatten()
        self.fc = nn.Conv1d(in_channels = 512, out_channels = 257, kernel_size = 1, stride = 1)
        
    def forward(self, x):
        # Encode
        x1 = self.input_layer(x) + self.input_skip(x)

        # x2 = self.squeeze_excite1(x1)
        x2 = self.residual_conv1(x1)

        # x3 = self.squeeze_excite2(x2)
        x3 = self.residual_conv2(x2)

        # x4 = self.squeeze_excite3(x3)
        x4 = self.residual_conv3(x3)
        x5 = self.aspp_bridge(x4)
        
        x6 = self.attn1(x3, x5)
        # x6 = self.upsample1(x6)
        x6 = torch.cat([x6, x3], dim=1)
        x6 = self.up_residual_conv1(x6)
        
        x7 = self.attn2(x2, x6)
        # x7 = self.upsample2(x7)
        x7 = torch.cat([x7, x2], dim=1)
        x7 = self.up_residual_conv2(x7)
        
        # x8 = self.attn3(x1, x7)
        # # x8 = self.upsample3(x8)
        # x8 = torch.cat([x8, x1], dim=1)
        # x8 = self.up_residual_conv3(x8)

        x9 = self.aspp_out(x7)
        output = self.fc(x9)
        return self.flat(output)

In [18]:
model = CNN()
optimizer = optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-12, verbose=True)

trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

Epoch : [0] Train loss : [0.06406954005360603] Val Score : [0.4410480349344978])
counter : set 0 min loss : 0.06406954005360603
Epoch : [1] Train loss : [0.05592035204172134] Val Score : [0.45064377682403434])
counter : set 0 min loss : 0.05592035204172134
Epoch : [2] Train loss : [0.047380104660987854] Val Score : [0.46218487394957986])
counter : set 0 min loss : 0.047380104660987854
Epoch : [3] Train loss : [0.04523633345961571] Val Score : [0.4796747967479675])
counter : set 0 min loss : 0.04523633345961571
Epoch : [4] Train loss : [0.043872532248497007] Val Score : [0.4817813765182186])
counter : set 0 min loss : 0.043872532248497007
Epoch : [5] Train loss : [0.03346921056509018] Val Score : [0.4980392156862745])
counter : set 0 min loss : 0.03346921056509018
Epoch : [6] Train loss : [0.032176388427615166] Val Score : [1.0])
counter : set 0 min loss : 0.032176388427615166
Epoch : [7] Train loss : [0.025649922713637353] Val Score : [1.0])
counter : set 0 min loss : 0.025649922713637

In [19]:
model = CNN()
model.load_state_dict(torch.load('./best_model.pth'))
model.eval()

CNN(
  (input_layer): Sequential(
    (0): Conv1d(257, 512, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
  )
  (input_skip): Sequential(
    (0): Conv1d(257, 512, kernel_size=(1,), stride=(1,))
  )
  (squeeze_excite1): Squeeze_Excite_Block(
    (avg_pool): AdaptiveAvgPool1d(output_size=1)
    (fc): Sequential(
      (0): Linear(in_features=512, out_features=32, bias=False)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=512, bias=False)
      (3): Sigmoid()
    )
  )
  (residual_conv1): ResConv(
    (conv_block): Sequential(
      (0): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
    )
    (conv_skip): Sequential(
      (0): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
 

## Prediction

In [20]:
test_dataset = CustomDataset(test, False)
test_loader = DataLoader(test_dataset, batch_size=BATCH, shuffle=False)

In [47]:
def prediction(model, test_loader, device, thr=0.9993):
    model.to(device)
    model.eval()
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    pred = []
    with torch.no_grad():
        for x in iter(test_loader):
            x = x.float().to(device)
            _x = model(x)
            log_target = F.log_softmax(_x, dim=1)
            log_input = F.log_softmax(x.reshape(-1,257), dim=1)
                
            diff = cos(log_input, log_target).cpu().tolist()
            print(diff)
            batch_pred = np.where(np.array(diff) > thr, 0, 1).tolist()
            pred += batch_pred
    return pred

In [48]:
preds = prediction(model, test_loader, device)
sum(preds)

[0.9992483258247375, 0.9993578791618347, 0.9994314908981323, 0.9983837604522705, 0.9993669986724854, 0.9996156692504883, 0.9995800852775574, 0.9994066953659058, 0.9989098906517029, 0.9995006918907166, 0.9993021488189697, 0.9992926120758057, 0.9990189075469971, 0.9988436698913574, 0.9995005130767822, 0.9991111159324646, 0.9994137287139893, 0.9992807507514954, 0.9992237091064453, 0.9983579516410828, 0.9985300302505493, 0.9991097450256348, 0.9994000196456909, 0.9986343383789062, 0.9989681839942932, 0.9990826845169067, 0.9989393949508667, 0.99910569190979, 0.9992388486862183, 0.9995608329772949, 0.9995982050895691, 0.9990841150283813, 0.9995185136795044, 0.9990791082382202, 0.999056339263916, 0.9994670152664185, 0.9995079040527344, 0.999281644821167, 0.9993642568588257, 0.9994605779647827, 0.9992635250091553, 0.999529242515564, 0.9992015361785889, 0.9994982481002808, 0.9990350604057312, 0.9997432231903076, 0.9995969533920288, 0.9992828369140625, 0.9994145035743713, 0.9986430406570435, 0.99

850

## Submission

In [49]:
submit = pd.read_csv(path+'sample_submission.csv')

In [50]:
submit['LABEL'] = preds
submit

,SAMPLE_ID,LABEL
0,TEST_0000,1
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,1
4,TEST_0004,0
...,...,...
1509,TEST_1509,1
1510,TEST_1510,1
1511,TEST_1511,0
1512,TEST_1512,0


In [51]:
submit.to_csv('submit.csv', index=False)

In [52]:
submit['LABEL'].sum()

850